In [81]:
import fitz
from haystack.components.builders import PromptBuilder
from openai import OpenAI
from env_params import openrouter_api_key
import json
from IPython.display import clear_output


quiz_generation_template = """Given the following text, create {{no_questions}} multiple choice quizzes in JSON format.
Each question should have 4 different options, and only one of them should be correct.
The questions should be {{question_level}}.
Each question should increase in difficulty.
Each question should be independent of the others.
Each question should be based on the text provided.
Each question should be clear and concise.
The options should be unambiguous.
Each option should begin with a letter followed by a period and a space (e.g., "a. option").
The question should also briefly mention the general topic of the text so that it can be understood in isolation.
Each question should not give hints to answer the other questions.
Include challenging questions, which require reasoning.

respond with JSON only, no markdown or descriptions.

example JSON format you should absolutely follow:
{"topic": "a sentence explaining the topic of the text",
 "questions":
  [
    {
      "question": "text of the question",
      "options": ["a. 1st option", "b. 2nd option", "c. 3rd option", "d. 4th option"],
      "right_option": "c"  # letter of the right option ("a" for the first, "b" for the second, etc.)
    }, ...
  ]
}


text:
{% for doc in documents %}{{ doc.content|truncate(1000000) }}{% endfor %}
"""


def get_question_level():
    got_answer = False
    while not got_answer:
        try:
            question_level = input("Enter the question level (easy (e), medium (m), hard (h)): ")
            if question_level.lower() in ['easy','medium','hard','e','m','h']:
                got_answer = True
            else:
                print("Please enter a valid question level")
        except ValueError:
            print("Please enter a valid question level")
    if question_level.lower() == 'easy' or question_level.lower() == 'e':
        question_level = "easy"
    elif question_level.lower() == 'medium' or question_level.lower() == 'm':
        question_level = "medium"
    else:
        question_level = "hard"
    return question_level

def read_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)
        text += page.get_text()
    return text

def get_list_of_documents(documents_paths):
    document_list = []
    for document in documents_paths:
        document_list.append({'content':read_pdf(document)})
    return document_list

def get_number_of_questions():
    got_answer = False
    while not got_answer:
        try:
            no_questions = int(input("Enter the number of questions: "))
            got_answer = True
        except ValueError:
            print("Please enter a number")
    return no_questions

def get_path_for_documents():
    got_answer = False
    while not got_answer:
        try:
            documents_paths = input("Enter the path of the documents separated by a space: ").split()
            got_answer = True
        except ValueError:
            print("Please enter a valid path")
    return documents_paths

def create_prompt():
    documents_paths = get_path_for_documents()
    no_questions = get_number_of_questions()
    question_level = get_question_level()
    documents = get_list_of_documents(documents_paths)
    builder = PromptBuilder(
    template=quiz_generation_template,
    required_variables=["documents","no_questions","question_level"]
    )
    return builder.run(documents=documents, no_questions=no_questions, question_level=question_level)

def call_openrouter(openrouter_api_key, prompt):
    print("Please wait a few seconds for the response (average is 20 seconds)")
    client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
    )
    completion = client.chat.completions.create(
    model="deepseek/deepseek-r1-distill-llama-70b:free",
    messages=[
        {
        "role": "user",
        "content": prompt['prompt']
        }
    ]
    )
    try:
        return completion.choices[0].message.content.replace('```json','').replace('```','').replace('```','')
    except:
        return completion.choices[0].message.content

import json
def answer_questions(response):
    correct_answer = 0
    response = json.loads(response)
    print(response['topic'])
    for question in response['questions']:
        applicable_options = ["a", "b", "c", "d"]
        got_answer = False
        while not got_answer:
            print(question['question'])
            for option in question['options']:
                print(option)
            answer = input("Enter the right option: ").lower()
            if answer in applicable_options:
                got_answer = True
            else:
                clear_output(wait = True)
                print('',flush=True)
                print("Please enter a valid option")
        clear_output(wait = True)
        print('',flush=True)
        if answer == question['right_option']:
            print("Correct!\n")
            correct_answer += 1
        else:
            print(f"Wrong! The right option is {question['right_option']}\n")

    
    print(f"Your score is {correct_answer}/{len(response['questions'])}")

In [82]:
prompt = create_prompt()
response = call_openrouter(openrouter_api_key, prompt)
answer_questions(response)




Wrong! The right option is b

Your score is 4/11


In [83]:

def answer_questions(response):
    correct_answer = 0
    response = json.loads(response)
    print(response['topic'])
    for question in response['questions']:
        applicable_options = ["a", "b", "c", "d"]
        got_answer = False
        while not got_answer:
            print(question['question'])
            for option in question['options']:
                print(option)
            answer = input("Enter the right option: ").lower()
            if answer in applicable_options:
                got_answer = True
            else:
                #clear_output(wait = True)
                #print('',flush=True)
                print("Please enter a valid option")
        #clear_output(wait = True)
        #print('',flush=True)
        if answer == question['right_option']:
            print("Correct!\n")
            correct_answer += 1
        else:
            print(f"Wrong! The right option is {question['right_option']}\n")

    
    print(f"Your score is {correct_answer}/{len(response['questions'])}")

answer_questions(response)

Négyzetgyökös egyenletek
Mik azok a négyzetgyökös egyenletek?
a. Olyan egyenletek, amelyekben az ismeretlen a négyzetgyök alatt szerepel
b. Olyan egyenletek, amelyekben az ismeretlen a gyökvonás után szerepel
c. Olyan egyenletek, amelyekben az ismeretlen a függvények között szerepel
d. Olyan egyenletek, amelyekben az ismeretlen a logaritmusok között szerepel
Correct!

Mivel kell kiküszöbölni a négyzetgyökös egyenleteket?
a. Négyzetre emeléssel
b. Logaritmusokkal
c. Függvényekkel
d. Szorzással
Correct!

Melyik a szükséges lépés az egyenlet előtti kikötések meghatározásához?
a. A négyzetgyök alatti kifejezésnek nemnegatívnak kell lennie
b. A gyökvonás utáni kifejezésnek negatívnak kell lennie
c. A gyökvonás utáni kifejezésnek nullának kell lennie
d. A gyökvonás utáni kifejezésnek pozitívnak kell lennie
Correct!

Hogyan ellenőrizhetjük, hogy a gyök valóban megoldása az eredeti egyenletnek?
a. Csak a kikötéseket kell meghatározni
b. Az egyenlet mindkét oldalának négyzetre emelése
c. A gyök

In [84]:
no_questions

NameError: name 'no_questions' is not defined

In [ ]:
example_data/square_root.pdf example_data/Arithmetic_and_algebra.pdf

In [70]:
json.loads(response.choices[0].message.content)#[-100:]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
print(response.choices[0].message.content)
response.choices[0].message.content.replace('```json','').replace('```','').replace('```','')
```


```json
{
  "topic": "Arithmetic and algebra concepts, including sets of numbers and intervals.",
  "questions": [
    {
      "question": "Which of the following represents the set of irrational numbers?",
      "options": [
        "a. ℕ",
        "b. ℤ",
        "c. ℚ",
        "d. ℚ∗"
      ],
      "right_option": "d"
    },
    {
      "question": "Which notation represents a closed interval from a to b?",
      "options": [
        "a. (a, b)",
        "b. [a, b)",
        "c. [a, b]",
        "d. (a, b]"
      ],
      "right_option": "c"
    }
  ]
}
```


Please wait a few seconds for the response (average is 20 seconds)


In [50]:
#response_save = response.copy()#
response = response.choices[0].message.content


In [ ]:
list_of_easy_questions = []
list_of_medium_questions = []
list_of_hard_questions = []
for i in json.loads(response):
        list_of_easy_questions.append(i)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [19]:
from IPython.display import clear_output


In [53]:
response

'```json\n{\n  "topic": "A 2020-as matematika tankönyv tartalma és szerkezete",\n  "questions": [\n    {\n      "question": "Mikor jelent meg a tankönyv?",\n      "options": ["a. 2017", "b. 2020", "c. 201'